# When we have to do log transformation?

## find threshold

### define functions

In [1]:
from data_pipeline.open_data_raw_material_price.core import OpenDataRawMaterialPrice
from data_pipeline.open_data_marine_weather.core import OpenDataMarineWeather
from data_pipeline.open_data_terrestrial_weather.core import OpenDataTerrestrialWeather
import pandas as pd
import functools
from sklearn.preprocessing import QuantileTransformer, PowerTransformer ,MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import skew


# transformer
def log_transform(df):
    return np.log1p(df)

def sqrt_transform(df):
    return np.sqrt(df)


log = log_transform
sqrt = sqrt_transform
standard = StandardScaler()

t_names = ['log', 'standard', 'None']
transformers = [log, standard, 'None']



def get_skews(df):
    return df.apply(lambda x : skew(x))

def sum_corr(df):
    # default: method=pearson, min_periods=1
    # method{‘pearson’, ‘kendall’, ‘spearman’} 
    corr = df.corr()
    return abs(corr['당일조사가격'].drop('당일조사가격')).sum()

def analyze_skew(df):
    return get_skews(df)

def analyze_coef(df):
    return sum_coef(df)

def transform(transformer, df):
    if isinstance(transformer, TransformerMixin):
        return pd.DataFrame(transformer.fit_transform(df), columns=df.columns)
    elif transformer == 'None':
        return df
    else: 
        return transformer(df)



def build_dataset(date="201908"):
    # load
    t = OpenDataTerrestrialWeather(
        date=date
    )
    t_df = t.clean(t.filter(t.input_df))

    m = OpenDataMarineWeather(
        date=date
    )
    m_df = m.clean(m.filter(m.input_df))

    p = OpenDataRawMaterialPrice(
        date=date
    )
    p_df = p.clean(p.filter(p.input_df))
    
    # merge
    w_df = pd.merge(
                t_df, m_df,
                how='inner', on="일시"
            )
    origin_df = pd.merge(
                    p_df, w_df, how="inner", left_on="조사일자", right_on="일시"
                ).drop("일시", axis=1).astype(dtype={"조사일자": "datetime64"})
    return origin_df

def split_xy(df):
    X = df.drop("당일조사가격" ,axis=1)
    y = df['당일조사가격'].rename('price')
    return X, y

def corr_xy(x, y):
    corr = pd.concat([x,y] ,axis=1).corr()
    return abs(corr['price']).drop('price').sum()

def search_transformers(column, X: pd.DataFrame, y: pd.Series):
    """
        iterate transformer for X and compare with y (corr_xy)
    """
    x = X[column]
    l_tx = list(map(functools.partial(transform, df=pd.DataFrame(x)), transformers))

    l_coef = list(map(functools.partial(corr_xy, y=y), l_tx))
    
    # find max coef and index 
    max_coef = max(l_coef)
    max_index = l_coef.index(max_coef)
    
    proper_transformer = t_names[max_index]
    return proper_transformer, max_coef
                
def iterate_x(y: pd.Series, X: pd.DataFrame):
    # iterate X
    return list(map(functools.partial(search_transformers, X=X, y=y), X.columns.tolist()))

def grid_search(X: pd.DataFrame, y: pd.Series):
    """
    return: result grid, pd DataFrame
    """
    l_ty = list(map(functools.partial(transform, df=pd.DataFrame(y)), transformers))
    
    # iterate y
    result = list(map(functools.partial(iterate_x, X=X), l_ty))
    return pd.DataFrame(result, columns=X.columns, index=t_names)

def get_final_df(df):
    sum_df = pd.DataFrame(np.array(df.values.tolist())[:, :, 1], df.index, df.columns).astype("float").sum(axis=1).rename("sum")
    transformer_df = pd.DataFrame(np.array(df.values.tolist())[:, :, 0], df.index, df.columns)
    return pd.concat([sum_df, transformer_df], axis=1)


# main: pipeline
def pipeline(date="201908"):
    origin_df = build_dataset(date=date)
    X, y = split_xy(numeric_df)
    result_df = grid_search(X, y)
    return get_final_df(result_df),sum_corr(numeric_df), get_skews(X)

In [10]:
result08, ori_corr08, skews08 = pipeline(date="201908")

2020-04-02 04:03:35:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-02 04:03:35:core:<INFO> Series([], dtype: int64)
2020-04-02 04:03:47:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-02 04:03:47:core:<INFO> Series([], dtype: int64)
2020-04-02 04:05:18:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'
2020-04-02 04:05:18:core:<INFO> no missing value at raw material price


In [14]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(ori_corr08)
result08

0.04754771873052942


,sum,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),평균 풍속(m/s)_y,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
log,0.047980,None,log,None,log,log,log,None,log,None,log,None,None,None,log,log,log,log,log,log,log
None,0.048665,log,None,log,log,log,None,None,log,None,None,None,None,None,None,log,log,log,log,None,log


In [18]:
result08.reset_index().rename(columns={"index": "price"})

,price,sum,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),평균 풍속(m/s)_y,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,log,0.047980,None,log,None,log,log,log,None,log,None,log,None,None,None,log,log,log,log,log,log,log
1,None,0.048665,log,None,log,log,log,None,None,log,None,None,None,None,None,None,log,log,log,log,None,log


In [12]:
skews08

평균기온(°C)         -0.449520
최저기온(°C)         -0.560311
최고기온(°C)         -0.641829
최대 풍속(m/s)        0.341646
평균 풍속(m/s)_x      0.750572
최소 상대습도(pct)      0.565822
평균 상대습도(pct)_x    0.673415
강수 계속시간(hr)       0.864804
일강수량(mm)          1.109568
평균 풍속(m/s)_y      0.097296
평균기압(hPa)        -1.146476
평균 상대습도(pct)_y   -1.212847
평균 기온(°C)        -0.860780
평균 수온(°C)        -0.919219
평균 최대 파고(m)       0.648147
평균 유의 파고(m)       0.735840
최고 유의 파고(m)       0.643821
최고 최대 파고(m)       0.427031
평균 파주기(sec)       0.761779
최고 파주기(sec)       0.650867
dtype: float64

In [19]:
result07, ori_corr07, skews07 = pipeline(date="201907")

2020-04-02 04:47:19:manager:<INFO> 1 files is loaded from public_data/open_data_terrestrial_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-02 04:47:19:core:<INFO> 강수 계속시간(hr)    1
dtype: int64
2020-04-02 04:47:28:manager:<INFO> 1 files is loaded from public_data/open_data_marine_weather/origin/csv/2014-2020.csv in s3 'production-bobsim'
2020-04-02 04:47:28:core:<INFO> Series([], dtype: int64)
2020-04-02 04:48:25:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201907.csv in s3 'production-bobsim'
2020-04-02 04:48:26:core:<INFO> no missing value at raw material price


In [20]:
print(ori_corr07)
result07.reset_index().rename(columns={"index": "price"})

0.05504804034174497


,price,sum,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),평균 풍속(m/s)_y,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,log,0.068223,None,log,log,None,None,log,log,log,log,log,log,log,log,log,log,log,None,log,log,log
1,None,0.061753,log,log,log,None,log,log,log,log,log,log,None,log,log,log,log,log,log,log,log,log


In [25]:
skews07.to_frame().T

,평균기온(°C),최저기온(°C),최고기온(°C),최대 풍속(m/s),평균 풍속(m/s)_x,최소 상대습도(pct),평균 상대습도(pct)_x,강수 계속시간(hr),일강수량(mm),평균 풍속(m/s)_y,평균기압(hPa),평균 상대습도(pct)_y,평균 기온(°C),평균 수온(°C),평균 최대 파고(m),평균 유의 파고(m),최고 유의 파고(m),최고 최대 파고(m),평균 파주기(sec),최고 파주기(sec)
0,0.165048,0.279966,-0.440246,0.529074,0.456861,-0.400158,-0.386126,0.957824,0.80754,0.191083,0.504193,-0.85356,0.459598,0.428036,0.601955,0.538537,1.22191,1.178716,0.777665,1.303977
